In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [3]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,1112428133,aRAV90,0.0,Full-time,Senior / Semi-Senior,Marketing,Bigbox,5.0,342.0,249.0,4.0,FEM,28.096610,41.0,MASC,12
1,1111481489,aRAV90,0.0,Full-time,Senior / Semi-Senior,Atención al Cliente,RANDSTAD,216.0,NaN,NaN,NaN,NaN,NaN,41.0,MASC,12
2,1112389598,aRAV90,0.0,Full-time,Senior / Semi-Senior,Producción,Gentile y Asociados Consultores,106.0,49.0,1012.0,31.0,MASC,30.400943,41.0,MASC,12
3,1112383515,aRAV90,0.0,Full-time,Senior / Semi-Senior,Calidad,Adecco - Spring Professional,155.0,27.0,168.0,5.0,MASC,34.179487,41.0,MASC,12
4,1112278394,aRAV90,0.0,Full-time,Senior / Semi-Senior,Corporate Finance / Banca Inversión,Adecco -Región Cuyo & Patagonia,212.0,155.0,340.0,12.0,MASC,38.044444,41.0,MASC,12


In [4]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,42.0,FEM,21,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,31.0,MASC,9,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,36.0,FEM,12,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,69.0,MASC,15,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,32.0,FEM,11,1


In [5]:
avisos = None
postulantes = None

In [6]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante','id'],axis=1,inplace=True)


In [11]:
print(test.shape)
test.head()

(100000, 13)


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,42.0,FEM,21
1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,31.0,MASC,9
2,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,NaN,NaN,NaN,NaN,36.0,FEM,12
3,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,69.0,MASC,15
4,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,NaN,NaN,NaN,NaN,32.0,FEM,11


In [12]:
print(postulaciones.shape)
postulaciones.head()

(19783189, 13)


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,Full-time,Senior / Semi-Senior,Marketing,Bigbox,5.0,342.0,249.0,4.0,FEM,28.096610,41.0,MASC,12
1,Full-time,Senior / Semi-Senior,Atención al Cliente,RANDSTAD,216.0,NaN,NaN,NaN,NaN,NaN,41.0,MASC,12
2,Full-time,Senior / Semi-Senior,Producción,Gentile y Asociados Consultores,106.0,49.0,1012.0,31.0,MASC,30.400943,41.0,MASC,12
3,Full-time,Senior / Semi-Senior,Calidad,Adecco - Spring Professional,155.0,27.0,168.0,5.0,MASC,34.179487,41.0,MASC,12
4,Full-time,Senior / Semi-Senior,Corporate Finance / Banca Inversión,Adecco -Región Cuyo & Patagonia,212.0,155.0,340.0,12.0,MASC,38.044444,41.0,MASC,12


In [13]:
test = pd.concat([test,postulaciones])

In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not test[col].dtype == 'O': continue
    print(col)
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])
test.head()

tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
TendenciaSexo
sexo


,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,MASC,NO_DECLARA,TendenciaSexo,prom_edad,fechanacimiento,sexo,nivel
0,1,1,30,448,102.0,NaN,NaN,NaN,2,NaN,42.0,1,21
1,1,1,30,448,102.0,NaN,NaN,NaN,2,NaN,31.0,2,9
2,1,1,30,448,102.0,NaN,NaN,NaN,2,NaN,36.0,1,12
3,1,3,170,3809,27.0,NaN,NaN,NaN,2,NaN,69.0,2,15
4,1,3,170,3809,27.0,NaN,NaN,NaN,2,NaN,32.0,1,11


In [15]:
test = test.fillna(-1)
postulaciones = test[100000:]
test = test[:100000]

In [16]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [17]:
postulaciones.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,4,120,506,5.0,342.0,249.0,4.0,0,28.096610,41.0,2,12
1,1,4,14,3157,216.0,-1.0,-1.0,-1.0,2,-1.000000,41.0,2,12
2,1,4,143,1775,106.0,49.0,1012.0,31.0,1,30.400943,41.0,2,12
3,1,4,25,203,155.0,27.0,168.0,5.0,1,34.179487,41.0,2,12
4,1,4,44,204,212.0,155.0,340.0,12.0,1,38.044444,41.0,2,12


In [18]:
test.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,1,30,448,102.0,-1.0,-1.0,-1.0,2,-1.0,42.0,1,21
1,1,1,30,448,102.0,-1.0,-1.0,-1.0,2,-1.0,31.0,2,9
2,1,1,30,448,102.0,-1.0,-1.0,-1.0,2,-1.0,36.0,1,12
3,1,3,170,3809,27.0,-1.0,-1.0,-1.0,2,-1.0,69.0,2,15
4,1,3,170,3809,27.0,-1.0,-1.0,-1.0,2,-1.0,32.0,1,11


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=50,n_jobs=-1,warm_start=True,criterion='entropy')
clf.fit(postulaciones,resultado)
y_pred=clf.predict_proba(test)



0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
5           0.0
6           0.0
7           0.0
8           0.0
9           0.0
10          0.0
11          0.0
12          0.0
13          0.0
14          0.0
15          0.0
16          0.0
17          0.0
18          0.0
19          0.0
20          0.0
21          0.0
22          0.0
23          0.0
24          0.0
25          0.0
26          0.0
27          0.0
28          0.0
29          0.0
           ... 
19783159    1.0
19783160    1.0
19783161    1.0
19783162    1.0
19783163    1.0
19783164    1.0
19783165    1.0
19783166    1.0
19783167    1.0
19783168    1.0
19783169    1.0
19783170    1.0
19783171    1.0
19783172    1.0
19783173    1.0
19783174    1.0
19783175    1.0
19783176    1.0
19783177    1.0
19783178    1.0
19783179    1.0
19783180    1.0
19783181    1.0
19783182    1.0
19783183    1.0
19783184    1.0
19783185    1.0
19783186    1.0
19783187    1.0
19783188    1.0
Name: sepostulo, Length:

In [23]:
test = pd.read_csv('Sources/test_final_100k.csv')
test.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [28]:
test['sepostulo'] = y_pred[:,1]
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_random.csv', index=False)

In [ ]:
y_pred[y_pred==1].shape

In [ ]:
test.head()